# 4. VBL-VA001 Dataset

In [2]:
import pandas as pd
import numpy as np

sampling_rate = 20*1000 # 20kHz
motor_speed = 3000 # 3000 RPM fixed

## 4.1 개별 데이터 확인
- 먼저 개별 데이터(파일)이 어떤 형태로 존재하는 지 확인해 본다.
- 확인 후, 데이터를 센서 단위로 잘라서 np.float32로 바꾸는 작업까지 해둔다.
> 개별 파일에서 x, y, z 를 추출하는 작업을 함수화 


![Sample Image](figs/4_sensor.png )

In [3]:
file_path = '/home/lilmae/Desktop/Rotray_Machine_Data/dataset/VBL-VA001/bearing/bearing_000_Ch08_100g_PE_Acceleration.csv'
data_pd = pd.read_csv(file_path, header=None)
data_pd.columns = ['time', 'x', 'y', 'z']

time_np = data_pd['time'].to_numpy(dtype='float32')
x_np = data_pd['x'].to_numpy(dtype='float32')
y_np = data_pd['y'].to_numpy(dtype='float32')
z_np = data_pd['z'].to_numpy(dtype='float32')

print(f'x_len : {len(x_np)}, y_len : {len(y_np)}, z_len : {len(z_np)}')

x_len : 99939, y_len : 99939, z_len : 99939


## 4.2 데이터 interpolation
- 서로 다른 sampling rate 를 가지는 데이터를 활용하기 때문에 이 sampling rate 를 맞추어 주는 것이 필요하다.
- target sampling rate 를 기준으로 데이터를 interpolation 할 수 있도록 한다.

In [4]:
start = time_np[0]
end = time_np[-1]
total = end-start
print(f'start time : {start}, end time : {end}, total_time : {total}')

# 각 기록 시간은 시작시간 기준이 아닌 종료 시간 기준인 것을 알 수 있다. 이를 시작시간 기준으로 바꾸어 interpolation을 용이하게 해준다.
time_np = time_np - start

start = time_np[0]
end = time_np[-1]
total = end-start
print(f'start time : {start}, end time : {end}, total_time : {total}')

start time : 0.00292899995110929, end time : 4.999981880187988, total_time : 4.9970526695251465
start time : 0.0, end time : 4.9970526695251465, total_time : 4.9970526695251465


In [5]:
from scipy.interpolate import interp1d

signal = x_np # 센서 하나씩 interpolation할 수 있도록 한다 : 예시 x-sensor
target_sampling = 10*1000 # 기준으로 정한 sampling rate : 예시 10kHz
target_time = np.linspace(0, end, target_sampling, endpoint=False) # target sampling rate에 맞도록 시간축 생성

interpolator = interp1d(time_np, signal, kind='linear')
interpolated_signal = interpolator(target_time)

## 4.3 전체 데이터를 순회해서 접근
- 전체 데이터를 순회하여 접근하고 데이터 파일과 클래스 정보를 확인하는 작업을 해준다.
    > 클래스가 unbalance 인 경우 6g 과 27g 의 하부 클래스가 존재함
- 이를 파일 이름, 파일 경로, 클래스 정보 로 데이터베이스화 한다.

In [ ]:
import os

vbl_root = os.path.join(os.getcwd(), 'dataset', 'VBL-VA001')

data ={
    'file_name' : [],
    'file_path' : [],
    'class' : [],
    'load' : [],
    'severity' : [],
    'sampling_rate' : sampling_rate,
    'speed' : motor_speed
}

for class_name in os.listdir(vbl_root):
    class_dir = os.path.join(vbl_root, class_name)
    
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)

        severity = None
        # 클래스가 unbalance 인 경우에는 하부 클래스 정보가 추가로 필요하다.
        if class_name == 'unbalance':
            unbalance_scale = file_name.split('_')[0]
            if unbalance_scale =='ub':
                unbalance_scale = file_name.split('_')[1]
            else:
                unbalance_scale = unbalance_scale[-2:]
            severity = unbalance_scale
        
        data['file_name'].append(file_name)
        data['file_path'].append(file_path)
        data['class'].append(class_name)
        data['severity'].append(severity)

data_pd = pd.DataFrame(data)

In [7]:
data_pd.head()

,file_name,file_path,class,severity,sampling_rate,speed
0,normal_125-2_Ch08_100g_PE_Acceleration.csv,/home/lilmae/Desktop/Rotray_Machine_Data/datas...,normal,None,20000,3000
1,normal_095_Ch08_100g_PE_Acceleration.csv,/home/lilmae/Desktop/Rotray_Machine_Data/datas...,normal,None,20000,3000
2,normal_007-2_Ch08_100g_PE_Acceleration.csv,/home/lilmae/Desktop/Rotray_Machine_Data/datas...,normal,None,20000,3000
3,normal_194_Ch08_100g_PE_Acceleration.csv,/home/lilmae/Desktop/Rotray_Machine_Data/datas...,normal,None,20000,3000
4,normal_z_005-2_Ch08_100g_PE_Acceleration.csv,/home/lilmae/Desktop/Rotray_Machine_Data/datas...,normal,None,20000,3000


## 4.4 전체 함수화
1. `generate_vbl_df` : 전체 데이터를 순회해 pd.DataFrame 으로 만드는 함수
2. `open_vbl_file` : 개별 데이터를 열어 time,x,y,z 의 np.array 로 추출해 주는 함수
3. `interpol_vbl` :센서 데이터를 지정하는 sampling rate로 interpolation 해주는 함수

In [8]:
import os
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# 1. generate_vbl_df : 전체 데이터를 순회해 pd.DataFrame 으로 만드는 함수
def generate_vbl_df(vbl_root):

    sampling_rate = 20*1000 # 20kHz
    motor_speed = 3000 # 3000 RPM fixed

    data ={
        'file_name' : [],
        'file_path' : [],
        'class' : [],
        'severity' : [],
        'sampling_rate' : sampling_rate,
        'speed' : motor_speed
    }

    for class_name in os.listdir(vbl_root):
        class_dir = os.path.join(vbl_root, class_name)
        
        for file_name in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file_name)

            severity = None
            # 클래스가 unbalance 인 경우에는 하부 클래스 정보가 추가로 필요하다.
            if class_name == 'unbalance':
                unbalance_scale = file_name.split('_')[0]
                if unbalance_scale =='ub':
                    unbalance_scale = file_name.split('_')[1]
                else:
                    unbalance_scale = unbalance_scale[-2:]
                severity = unbalance_scale
            
            data['file_name'].append(file_name)
            data['file_path'].append(file_path)
            data['class'].append(class_name)
            data['severity'].append(severity)

    data_pd = pd.DataFrame(data)

    return data_pd

# 2. open_vbl_file : 개별 데이터를 열어 time,x,y,z 의 np.array 로 추출해 주는 함수
def open_vbl_file(file_path):
    data_pd = pd.read_csv(file_path, header=None)
    data_pd.columns = ['time', 'x', 'y', 'z']

    time_np = data_pd['time'].to_numpy(dtype='float32')
    x_np = data_pd['x'].to_numpy(dtype='float32')
    y_np = data_pd['y'].to_numpy(dtype='float32')
    z_np = data_pd['z'].to_numpy(dtype='float32')
    
    return time_np, x_np, y_np, z_np

# 3. interpol_vbl :센서 데이터를 지정하는 sampling rate로 interpolation 해주는 함수
def interpol_vbl(signal_np, time_np, target_sampling):
    
    sampling_rate = 20*1000 # 20kHz
    
    start = time_np[0]

    # 각 기록 시간은 시작시간 기준이 아닌 종료 시간 기준인 것을 알 수 있다. 이를 시작시간 기준으로 바꾸어 interpolation을 용이하게 해준다.
    time_np = time_np - start
    end = time_np[-1]

    target_sampling = 10*1000 # 기준으로 정한 sampling rate : 예시 10kHz
    target_time = np.linspace(0, end, target_sampling, endpoint=False) # target sampling rate에 맞도록 시간축 생성

    interpolator = interp1d(time_np, signal_np, kind='linear')
    interpolated_signal = interpolator(target_time)
    
    return interpolated_signal

In [ ]:
def load_vbl(vbl_root):

    sampling_rate = 20*1000 # 20kHz
    motor_speed = 3000 # 3000 RPM fixed

    file_name_list = []
    file_path_list = []
    class_name_list = []
    severity_list = []

    for class_name in os.listdir(vbl_root):
        class_dir = os.path.join(vbl_root, class_name)
        
        for file_name in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file_name)

            severity = None
            # 클래스가 unbalance 인 경우에는 하부 클래스 정보가 추가로 필요하다.
            if class_name == 'unbalance':
                unbalance_scale = file_name.split('_')[0]
                if unbalance_scale =='ub':
                    unbalance_scale = file_name.split('_')[1]
                else:
                    unbalance_scale = unbalance_scale[-2:]
                severity = unbalance_scale
            
            file_name_list.append(file_name)
            file_path_list.append(file_path)
            class_name_list.append(class_name)
            severity_list.append(severity)

    vbl_dataset = raw_dataset()
    vbl_dataset.file_name = file_name_list
    vbl_dataset.file_path = file_path_list
    vbl_dataset.class_name = class_name_list
    sampling_rate_list = [sampling_rate for _ in range(len(file_name_list))]
    vbl_dataset.sampling_rate = sampling_rate_list
    speed_list = [motor_speed for _ in range(len(file_name_list))]
    vbl_dataset.speed = speed_list

    return vbl_dataset

# 함수동작 확인

In [9]:
import os

fault_vbl_root = os.path.join(os.getcwd(), 'dataset', 'VBL-VA001')
target_sampling = 10*1000

data_pd = generate_vbl_df(fault_vbl_root)

for file_path in data_pd['file_path']:
    time_np, x_np, y_np, z_np = open_vbl_file(file_path)
    
    interpolated_signal = interpol_vbl(x_np, time_np, target_sampling)
    
print('NO Problem')

NO Problem
